In [7]:
import polars as pl
import pandas as pd
import re  
from time import time 
from collections import defaultdict

import spacy

In [8]:
!python -m spacy download ru_core_news_sm

/usr/bin/python: No module named spacy


In [9]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(text):
    if text is None:
        return ''
    tokens = tokenizer.tokenize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens
    # return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /home/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# nlp = spacy.load("ru_core_news_sm",disable=['ner', 'parser'])
# from tqdm import tqdm
# def preprocess_text2(text):
#     brief_cleaning = [re.sub("[^A-Za-zА-Яа-я']+", ' ', str(row)).lower() for row in text]

#     def cleaning(doc):
#         txt = [token.lemma_ for token in doc if not token.is_stop]
#         # return ' '.join(txt)
#         return txt
#     t = time()
#     txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000,n_process=14)]

#     print(round((time() - t) / 60, 2))
#     return txt
    

In [11]:
automarkup = pl.read_parquet('../train_data/automarkup.parquet')

FileNotFoundError: No such file or directory (os error 2): ../train_data/automarkup.parquet

In [ ]:
query =automarkup['query'].to_list()
del automarkup

In [ ]:
import gc
gc.collect()

In [ ]:
query[:10]

In [ ]:
preprocess_txt = list(map(lambda x: preprocess_text(x),query))

In [ ]:
# preprocess_txt2 = preprocess_text2(query)

In [ ]:
preprocess_txt2[:10]

In [ ]:
def make_markov_model_2(txt_list):
    markov_model = {}
    for req in txt_list:
        if len(req) == 0:
            continue
        current = req[0]
        for word in req[1:]:
            if current not in markov_model:
                markov_model[current] = {}
                markov_model[current][word] = 1
            else:
                if word in markov_model[current]:
                    markov_model[current][word] += 1
                else:
                    markov_model[current][word] = 1
            current = word
            
    
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model


In [ ]:
model = make_markov_model_2(preprocess_txt)

In [ ]:
# model1 =make_markov_model_2(preprocess_txt2)

In [ ]:
import random
def generate_supp(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        if curr_state not in markov_model:
            break
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(markov_model[curr_state].values()))
        
        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story

In [ ]:
for i in range(20):
    # print(str(i)+". ", generate_supp(model1, start="рецепт", limit=4))

In [ ]:
for i in range(20):
    print(str(i)+". ", generate_supp(model, start="рецепт", limit=4))

In [ ]:
from collections import defaultdict

def generate_supp3(markov_model, start, now=0, limit=10, result=None):
    if result is None:
        result =start +' '
    if now>= limit:
        return [result]
    if start not in markov_model:
        return [result]
    next_state = sorted(list(markov_model[start].keys()), key=lambda x: markov_model[start][x],reverse=True)
    sum_result=[]
    max_n=2
    for n,i in enumerate(next_state):
        if i in result:
            max_n+=1
            continue 
            
        sum_result+=generate_supp3(markov_model,i, now+1,limit, result + i+' ')
        if n>=max_n:
            break
        
    return sum_result
        

In [ ]:
generate_supp3(model,'рецепт',limit=3)

In [ ]:
# generate_supp3(model1,'рецепт',limit=3)

In [ ]:
import random
def generate_supp4(markov_model, limit=100, start='my god'):
    n = 0
    curr_state = start
    next_state = None
    story = ""
    story+=curr_state+" "
    while n<limit:
        if curr_state not in markov_model:
            break
        next_state = random.choices(list(markov_model[curr_state].keys()),
                                    list(map(lambda x: x**2,markov_model[curr_state].values())))
        
        curr_state = next_state[0]
        story+=curr_state+" "
        n+=1
    return story


In [ ]:
for i in range(20):
    print(str(i)+". ", generate_supp4(model, start="рецепт", limit=5))

In [ ]:
# for i in range(20):
    print(str(i)+". ", generate_supp4(model1, start="рецепт", limit=5))